In [ ]:
from util import *
import scipy.stats as stats
def compute_z_interval_old(j, n, d, O_obs, eps, neps, a, c, zk, minusO, x_zk):
  #print(len(O), len(minusO))
  I_d = np.identity(d)
  eT_minusO = np.zeros((1, n))
  eT_minusO[:,minusO] = 1

  #print(X)
  #print(x)
  
  eT_mean_minusO = np.kron(I_d, eT_minusO)/len(minusO)
  #print(np.dot(eT_mean_minusO, x), np.mean(X[minusO], axis = 0))

  e_j = np.zeros((1, n))
  e_j[:,j] = 1
  temp = np.kron(I_d, e_j) - eT_mean_minusO
  Xj_meanXminusO = np.dot(temp, x_zk)
  #print(Xj_meanXminusO, X[j] - np.mean(X[minusO], axis = 0))

  S = np.sign(Xj_meanXminusO)
  #print(S)
  B = np.multiply(S, temp)


  interval = []

  for j in range(n):
    #euclidean distance
    A1_sparse = []
    A2_sparse = []
    for i in range(n):
      e = np.zeros((1,n))
      e[0][j] = 1
      if i != j:
        e[0][i] = -1
      else:
        e[0][j] = 0
      Id_x_e = np.kron(I_d, e)
      Id_x_e_sparse = csr_matrix(Id_x_e)

      
      A_sparse = Id_x_e_sparse.T @ Id_x_e_sparse
      #sparsity = 1.0 - ( np.count_nonzero(A) / float(A.size) )
      #print(sparsity)
      
      if i in neps[j]:
       
        A1_sparse.append(A_sparse)
      else:
        
        A2_sparse.append(A_sparse)
    b = eps*eps
    b = np.array(b).reshape(1, 1)
    #print(a.shape, c.shape, z.shape)
    p1 = np.array([a.T @ (Ai @ a) for Ai in A1_sparse]).reshape(1, -1, 1)
    q1 = np.array([c.T @ (Ai @ a) + a.T @ (Ai @ c) for Ai in A1_sparse]).reshape(1,-1,1)
    t1 = np.array([c.T @ (Ai @ c) - b for Ai in A1_sparse]).reshape(1,-1,1)

    #print("p1", p1)
    p2 = -1*(np.array([a.T @ (Ai @ a) for Ai in A2_sparse])).reshape(1, -1, 1)
    q2 = -1*(np.array([c.T @ (Ai @ a) + a.T @ (Ai @ c) for Ai in A2_sparse])).reshape(1,-1,1)
    t2 = -1*(np.array([c.T @ (Ai @ c) - b for Ai in A2_sparse])).reshape(1,-1,1)
    #print("p2", p2)
    
            
    for k in range(len(A1_sparse)):
      res = solve_quadratic_inequality(p1[0][k][0], q1[0][k][0], t1[0][k][0])
      if res == "No solution":
        print(p1[0][k][0], q1[0][k][0], t1[0][k][0])
      else:
        interval.append(res)
                
      

    for k in range(len(A2_sparse)):
      res = solve_quadratic_inequality(p2[0][k][0], q2[0][k][0], t2[0][k][0])
      if res == "No solution":
        print(p2[0][k][0], q2[0][k][0], t2[0][k][0])
      else:
        interval.append(res)


   
  Ba = np.dot(B, a)
  Bc = np.dot(B, c)
  Vminus = -np.inf
  Vplus = np.inf
    #print(Ba.shape, Bc.shape)
  for j in range (Ba.shape[0]):
    if -1e-10 <= Ba[j][0] <= 1e-10:
      if not (-Bc[j][0] <= 0):
        print("error3")
    elif Ba[j][0] > 0:
      Vminus = max(Vminus, -Bc[j][0]/Ba[j][0])

    else:
      Vplus = min(Vplus, -Bc[j][0]/Ba[j][0])
  #print(Vminus, Vplus
  interval.append([(Vminus, Vplus)])
  #(V1, V2, V3, V4)

  if len(interval) == 0:
    print("empty interval")
    return None
  #print(interval

  intersection = find_intersection_of_all_unions(interval)
  return intersection, S

def compute_z_interval_fix(j_test, n, d, O_obs, eps, neps, a, c, zk, minusO, x_zk):
  #print(len(O), len(minusO))
  I_d = np.identity(d)
  
  trunc_interval = [(-np.inf, np.inf)]

  for j in range(n):
    #euclidean distance
    A1_sparse = []
    A2_sparse = []
    for i in range(n):
      e = np.zeros((1,n))
      e[0][j] = 1
      if i != j:
        e[0][i] = -1
      else:
        continue
      
      Id_x_e = np.kron(I_d, e)
      Id_x_e_sparse = csr_matrix(Id_x_e)
      A_sparse = Id_x_e_sparse.T @ Id_x_e_sparse
      #sparsity = 1.0 - ( np.count_nonzero(A) / float(A.size) )
      #print(sparsity)
      
      if i in neps[j]:
       
        A1_sparse.append(A_sparse)
      else:
        
        A2_sparse.append(A_sparse)
        
    b = eps*eps
    b = np.array(b).reshape(1, 1)
    #print(a.shape, c.shape, z.shape)
    p1 = np.array([a.T @ (Ai @ a) for Ai in A1_sparse]).reshape(1, -1, 1)
    q1 = np.array([c.T @ (Ai @ a) + a.T @ (Ai @ c) for Ai in A1_sparse]).reshape(1,-1,1)
    t1 = np.array([c.T @ (Ai @ c) - b for Ai in A1_sparse]).reshape(1,-1,1)

    #print("p1", p1)
    p2 = -1*(np.array([a.T @ (Ai @ a) for Ai in A2_sparse])).reshape(1, -1, 1)
    q2 = -1*(np.array([c.T @ (Ai @ a) + a.T @ (Ai @ c) for Ai in A2_sparse])).reshape(1,-1,1)
    t2 = -1*(np.array([c.T @ (Ai @ c) - b for Ai in A2_sparse])).reshape(1,-1,1)
    #print("p2", p2)

            
    for k in range(len(A1_sparse)):
      res = solve_quadratic_inequality(p1[0][k][0], q1[0][k][0], t1[0][k][0])
      if res == "No solution":
        print(p1[0][k][0], q1[0][k][0], t1[0][k][0])
      else:
        trunc_interval = interval_intersection(trunc_interval,res)
                
      

    for k in range(len(A2_sparse)):
      res = solve_quadratic_inequality(p2[0][k][0], q2[0][k][0], t2[0][k][0])
      if res == "No solution":
        print(p2[0][k][0], q2[0][k][0], t2[0][k][0])
      else:
        trunc_interval = interval_intersection(trunc_interval,res)


 

  eT_minusO = np.zeros((1, n))
  eT_minusO[:,minusO] = 1

  #print(X)
  #print(x)
  
  eT_mean_minusO = np.kron(I_d, eT_minusO)/len(minusO)
  #print(np.dot(eT_mean_minusO, x), np.mean(X[minusO], axis = 0))

  e_j = np.zeros((1, n))
  e_j[:, j_test] = 1
  temp = np.kron(I_d, e_j) - eT_mean_minusO
  Xj_meanXminusO = np.dot(temp, x_zk)
  #print(Xj_meanXminusO, X[j] - np.mean(X[minusO], axis = 0))

  S = np.sign(Xj_meanXminusO)
  #print(S)
  B = np.multiply(S, temp)
  Ba = np.dot(B, a)
  Bc = np.dot(B, c)
 
  #print(Ba.shape, Bc.shape)
  for j in range (Ba.shape[0]):
    res = solve_quadratic_inequality(0, -Ba[j][0], -Bc[j][0])
    trunc_interval = interval_intersection(trunc_interval,res)
  return trunc_interval, S


def run_sioc(n, d, minpts, eps):
  X, u, Sigma = generate(n, d)

  label, neps = dbscan_sk(X, minpts, eps)
  label = np.array(label)
  O = [i for i in range(label.shape[0]) if label[i] == -1]
    #print(O)
  if len(O) == 0 or len(O) == n:
    return None
  #test statistic

  j = np.random.choice(O)
  #print(neps)
  #print(X[j])
  minusO = [i for i in range(label.shape[0]) if label[i] != -1]
  #print(len(O), len(minusO))
  eT_minusO = np.zeros((1, label.shape[0]))
  eT_minusO[:,minusO] = 1
  x = vec(X)
  #print(X)
  #print(x)
  I_d = np.identity(d)
  eT_mean_minusO = np.kron(I_d, eT_minusO)/(n - len(O))
  #print(np.dot(eT_mean_minusO, x), np.mean(X[minusO], axis = 0))

  e_j = np.zeros((1, n))
  e_j[:,j] = 1
  temp = np.kron(I_d, e_j) - eT_mean_minusO
  Xj_meanXminusO = np.dot(temp, x)
  #print(Xj_meanXminusO, X[j] - np.mean(X[minusO], axis = 0))

  S = np.sign(Xj_meanXminusO)
 

  etaT = np.dot(S.T, temp)/d
  eta = np.transpose(etaT)
  #print("eta", eta.shape)
  #print("x", x.shape)

  etaTx = np.dot(etaT, x)

  etaT_Sigma_eta=np.dot(np.dot(eta.T, Sigma), eta)
  #print(etaT_Sigma_eta)


  a = np.dot(np.dot(Sigma, eta), np.linalg.inv(etaT_Sigma_eta))
  c = np.dot(np.identity(int(n*d)) - np.dot(a,eta.T), x)
  z = etaTx
  #print("z", z)
  #print(z)
  intersection1, S1 = compute_z_interval_fix(j, n, d, O, eps, neps, a, c, z, minusO, x)
  intersection2, S2 = compute_z_interval_old(j, n, d, O, eps, neps, a, c, z, minusO, x)
  #print(interval)
  #print(intersection)
  if not(np.allclose(np.array(intersection1), np.array(intersection2))):
    print(intersection1)
    print(intersection2)
    if not(np.array_equal(np.array(S1), np.array(S2))):
      print(S1)
      print(S2)
  z = etaTx[0][0]
  mu = 0

  cdf = pivot_with_specified_interval(intersection1, eta, z, Sigma, mu)
  if cdf is None:
    return None
  selective_p_value = 2 * min(cdf, 1 - cdf)
  #print(selective_p_value)
  if selective_p_value == 0 or selective_p_value is None:
    print("intersection, eta, z, etaT_Sigma_eta, mu",intersection1, z, etaT_Sigma_eta, mu)
  return selective_p_value


  

def run_wrapper(args):
    n, d, minpts, eps = args
    return run_sioc(n, d, minpts, eps)


if __name__ == '__main__':
    import numpy as np
    from scipy.stats import matrix_normal
    from util import *
    from multiprocessing import Pool
    from tqdm import tqdm
    import matplotlib.pyplot as plt
    import scipy.stats as stats

   

    max_iteration = 500
    n = 50
    d = 10
    minpts = 10     
    eps = 3
    Alpha = 0.05
    for i in range(100):
       
        p = run_sioc(n, d, minpts, eps)
        print(i, p)


0 0.25726864183957177
1 0.24019421365670812
2 0.007931104804881881
3 0.10673331688351051
4 0.7710753912447026
5 0.2723929262163258
6 0.7000917616684508
7 0.09183434737145656
8 0.8467443499208825
9 0.4075815231078506
10 0.35064235927254245
11 0.926880252178429
12 0.0384418831624979
13 0.7746630070693958
14 0.4940901563905495
15 0.4192121065263145
16 0.10792128165534254
17 0.5741912881793403
18 0.8370986674206267
19 0.37670768399395427
20 0.4051143372043285
21 0.19051876307739937
22 0.5204053388635983
23 0.012897583126961765
24 0.706298233189073
25 0.6536424996881818
26 0.17522591062595097
27 0.1657723254516851
28 0.7636967406074716
29 0.13299938896077476
30 None
31 0.30740617061097786
32 0.9677029433321974
33 0.2724047998424244
34 0.6681728107989937
35 0.3767032367071317
36 0.13198856494863387
37 0.49298106391324226
38 0.41402285848640275
39 0.7726589554625276
40 0.7940732449126762
41 0.6570427850083769
42 0.049096651158533365
43 0.6227748620681521
44 None
45 None
46 0.7097595104355774


In [2]:

import numpy as np
from scipy.stats import matrix_normal
from util import *
from multiprocessing import Pool
from tqdm import tqdm
import matplotlib.pyplot as plt
import scipy.stats as stats
def run_parametric_dbscan(j, n, d, O_obs, S_obs, minpts, eps, threshold, a, c):
  
  zk = - threshold
  final_itv = []  
  detected_itv = []  
  while zk < threshold:
    zk += 0.0001
    
    for i in range(len(detected_itv)):
        if detected_itv[i][0] <= etaTx[0][0] <= detected_itv[i][1]:
          print(detected_itv[i])
          print("O", np.array_equal(np.sort(setofOutliers), np.sort(O_obs)))
          print("sign", np.array_equal(S, S_obs))
        if detected_itv[i][0] <= zk <= detected_itv[i][1]:
            zk = detected_itv[i][1] + 0.0001
            detected_itv = detected_itv[i:]
            break
    if zk > threshold:
        break
    x_zk = a*zk + c
    X_zk = unvec(x_zk, n, d)
    label_zk, neps_zk = dbscan_sk(X_zk, minpts, eps)
    setofOutliers =  [i for i in range(len(label_zk)) if label_zk[i] == -1]
    minusO_zk = [i for i in range(len(label_zk)) if label_zk[i] != -1]

    intersection, S = compute_z_interval_fix(j, n, d, O_obs, eps, neps_zk, a, c, zk, minusO_zk, x_zk)

    detected_itv = interval_union(detected_itv, intersection)  #[[zk;...], [zk+1,...]]
    
    if not(np.array_equal(np.sort(setofOutliers), np.sort(O_obs)) and np.array_equal(S, S_obs)):
        continue
    final_itv = interval_union(final_itv, intersection)
        
     

    #print(zk)
  return final_itv
def compute_p_value(intervals, etaT_Y, etaT_Sigma_eta):
    denominator = 0
    numerator = 0

    for i in intervals:
        leftside, rightside = i
        if leftside <= etaT_Y <= rightside:
            numerator = (
                denominator
                + mp.ncdf(etaT_Y / np.sqrt(etaT_Sigma_eta))
                - mp.ncdf(leftside / np.sqrt(etaT_Sigma_eta))
            )
        denominator += mp.ncdf(rightside / np.sqrt(etaT_Sigma_eta)) - mp.ncdf(
            leftside / np.sqrt(etaT_Sigma_eta)
        )
    if denominator == 0:
        print("denominator error")
        return None
    cdf = float(numerator / denominator)
    return 2 * min(cdf, 1 - cdf)
def run_parametric(n, d, minpts, eps):

  X, u, Sigma = generate(n,d)

  threshold = 20
  
  label, neps = dbscan_sk(X, minpts, eps)
  label = np.array(label)
  O = [i for i in range(label.shape[0]) if label[i] == -1]
    #print(O)
  if len(O) == 0 or len(O) == n:
    return None
  #test statistic

  j = np.random.choice(O)
  #print(X[j])
  minusO = [i for i in range(label.shape[0]) if label[i] != -1]
  #print(len(O), len(minusO))
  eT_minusO = np.zeros((1, label.shape[0]))
  eT_minusO[:,minusO] = 1
  x = vec(X)
  #print(X)
  #print(x)
  I_d = np.identity(d)
  eT_mean_minusO = np.kron(I_d, eT_minusO)/(n - len(O))
  #print(np.dot(eT_mean_minusO, x), np.mean(X[minusO], axis = 0))

  e_j = np.zeros((1, n))
  e_j[:,j] = 1
  temp = np.kron(I_d, e_j) - eT_mean_minusO
  Xj_meanXminusO = np.dot(temp, x)
  #print(Xj_meanXminusO, X[j] - np.mean(X[minusO], axis = 0))

  S_obs = np.sign(Xj_meanXminusO)
  #print(S)
  

  etaT = np.dot(S_obs.T, temp)/d
  eta = np.transpose(etaT)
  #print("eta", eta.shape)
  #print("x", x.shape)
  global etaTx
  etaTx = np.dot(etaT, x)
  #print(etaTx)
  etaT_Sigma_eta=np.dot(np.dot(eta.T, Sigma), eta)
  #print(etaT_Sigma_eta)
  a = np.dot(np.dot(Sigma, eta), np.linalg.inv(etaT_Sigma_eta))
  c = np.dot(np.identity(int(n*d)) - np.dot(a,eta.T), x)
  print("oc", S_obs, O, compute_z_interval_fix(j, n, d, O, eps, neps, a, c, etaTx[0][0], minusO, x))
  
  para_interval = run_parametric_dbscan(j, n, d, O, S_obs, minpts, eps, threshold, a, c)
  
  print(etaTx, para_interval)
  

  selective_p_value = compute_p_value(para_interval, etaTx[0][0], etaT_Sigma_eta[0][0])
  if selective_p_value is None:
    return None
  
  return selective_p_value





if __name__ == '__main__':

    max_iteration = 500
    n = 50
    d = 10
    minpts = 10
    eps = 3
    Alpha = 0.05
    for _ in range(10):
        p = run_parametric(n, d, minpts, eps)
        print(p)
   

oc [[ 1.]
 [-1.]
 [-1.]
 [ 1.]
 [ 1.]
 [-1.]
 [ 1.]
 [-1.]
 [-1.]
 [ 1.]] [0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 13, 15, 17, 18, 20, 21, 23, 26, 27, 28, 29, 30, 31, 33, 34, 35, 36, 37, 38, 39, 40, 41, 43, 44, 45, 46, 47, 49] ([(np.float64(0.90780921), np.float64(1.02196786))], array([[ 1.],
       [-1.],
       [-1.],
       [ 1.],
       [ 1.],
       [-1.],
       [ 1.],
       [-1.],
       [-1.],
       [ 1.]]))
(np.float64(0.90780921), np.float64(1.02196786))
O True
sign True
(np.float64(0.90780921), np.float64(1.02196786))
O True
sign True
[[0.9881115]] [(np.float64(0.90780921), np.float64(1.02196786)), (np.float64(1.02196786), np.float64(1.02413756)), (np.float64(1.02413756), np.float64(1.07841777)), (np.float64(1.07841777), np.float64(1.08214655)), (np.float64(1.08214655), np.float64(1.1273249)), (np.float64(1.1273249), np.float64(1.18320338)), (np.float64(1.18320338), np.float64(1.26315014)), (np.float64(1.26315014), np.float64(1.67950705)), (np.float64(1.67950705), np.float64(1.

In [4]:
print(10%15)

10


In [10]:
j = 3
e_j = np.zeros((1, n))
e_j[:, j] = 1
print(e_j)
a = 0
for j in range(n):
    a += 1

e_j = np.zeros((1, n))
e_j[:, j] = 1
print(e_j)
    
    

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 1.]]
